In [1]:
import numpy as np
import msprime
import tskit
import sys
sys.path.append("../")
from tsimpute import beagle_numba

In [2]:
N = 100
L = 1e6
rho = 1e-8
mu = 1e-8
ne = 1e4
ts = msprime.sim_mutations(
    msprime.sim_ancestry(
        N,
        sequence_length=L,
        discrete_genome=True,
        recombination_rate=rho,
        population_size=ne,
    ),
    rate=mu,
)
ts

In [3]:
ne = 1e6
err_rate = 1e-4
cm = beagle_numba.convert_to_cm(ts.sites_position)
trans_probs = beagle_numba.get_transition_probs(cm, h=ts.num_samples, ne=ne)
mismatch_probs = beagle_numba.get_mismatch_probs(ts.num_sites, error_rate=err_rate)

In [4]:
ref_h = ts.genotype_matrix(alleles=tskit.ALLELES_ACGT)
ref_h[:1000, 0] = tskit.MISSING_DATA
ref_h[2000:, 0] = tskit.MISSING_DATA
print(np.sum(ref_h[:, 0] == tskit.MISSING_DATA))
print(np.sum(ref_h[:, 1:] == tskit.MISSING_DATA))

1246
0


In [5]:
query_h = np.random.choice(a=np.arange(4, dtype=np.int32), size=ts.num_sites)
query_h[1000:1010] = tskit.MISSING_DATA
np.sum(query_h == tskit.MISSING_DATA)

10

In [6]:
beagle_numba.run_interpolation_beagle(
    ref_h=ref_h,
    query_h=query_h,
    pos_all=query_h,
)

/Users/shingzhan/Projects/tsimpute/notebooks/../tsimpute/beagle_numba.py:577: UserWarning: This function is experimental and not fully tested.
  warnings.warn("This function is experimental and not fully tested.", stacklevel=1)
/Users/shingzhan/Projects/tsimpute/notebooks/../tsimpute/beagle_numba.py:578: UserWarning: Check the reference and query haplotypes use the same allele encoding.
  warnings.warn(


(array([0, 3, 2, 1, 0, 3, 3, 0, 3, 0]),
 array([1.        , 0.99976955, 1.        , 0.99988492, 0.99999627,
        0.99999627, 1.        , 1.        , 1.        , 1.        ]))